In [ ]:
import numpy as np; print('numpy Version:', np.__version__)
import pandas as pd; print('pandas Version:', pd.__version__)
import xgboost as xgb; print('XGBoost Version:', xgb.__version__)
from sklearn.model_selection import train_test_split

In [ ]:
# read data
train = pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-nov-2021/test.csv")

In [ ]:
# train data
train = train.drop(['id'], axis=1)
train

In [ ]:
# test data
test = test.drop(["id"], axis=1)
test

In [ ]:
features_col = train.columns 
train[features_col].select_dtypes(exclude=['int64']).columns

In [ ]:
X = train[features_col]
y = train['target']

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.33, random_state=42)

### Dimention

In [ ]:
# check dimensions
print('X_train: ', X_train.shape, X_train.dtype, 'y_train: ', y_train.shape, y_train.dtype)
print('X_validation', X_validation.shape, X_validation.dtype, 'y_validation: ', y_validation.shape, y_validation.dtype)

# check the proportions
total = X_train.shape[0] + X_validation.shape[0]
print('X_train proportion:', X_train.shape[0] / total)
print('X_validation proportion:', X_validation.shape[0] / total)

In [ ]:
%%time

dtrain = xgb.DMatrix(X_train, label=y_train)
dvalidation = xgb.DMatrix(X_validation, label=y_validation)

In [ ]:
xgb_prams = {'learning_rate': 0.010278270092400477, 'reg_lambda': 92.72404085345757, 
 'reg_alpha': 4.3381312346360553e-07, 'subsample': 0.6098232053124079, 'colsample_bytree': 0.26782224124790555,
 'max_depth': 8, 'early_stopping_rounds': 389, 'n_estimators': 15000}

In [ ]:

# instantiate params
params = {}

# general params
general_params = {'silent': 1, 'reg_lambda': 92.72404085345757, 
 'reg_alpha': 4.3381312346360553e-07, 'subsample': 0.6098232053124079, 'colsample_bytree': 0.26782224124790555,
 'max_depth': 8, 'early_stopping_rounds': 389, 'n_estimators': 15000}
params.update(general_params)

# booster params
n_gpus = 1
eta = 0.010278270092400477
booster_params = {}

if n_gpus != 0:
    booster_params['tree_method'] = 'gpu_hist'
    booster_params['predictor'] = 'gpu_predictor'
    booster_params['n_gpus'] = n_gpus
    booster_params['eta'] = eta
params.update(booster_params)

# learning task params
learning_task_params = {'eval_metric': 'auc', 'objective': 'binary:logistic'}
params.update(learning_task_params)
print(params)

In [ ]:
# model training settings
evallist = [(dvalidation, 'validation'), (dtrain, 'train')]
num_round = 10

In [ ]:
%%time

bst = xgb.train(params, dtrain, num_round, evallist)

In [ ]:
# test.drop(["target"], axis=1)
test["target"] = np.nan
test

In [ ]:
# prepare test

dtest = xgb.DMatrix(data=test)

In [ ]:
test_preds = bst.predict(dtest)

In [ ]:
sample_submission = pd.read_csv("../input/tabular-playground-series-nov-2021/sample_submission.csv")

In [ ]:
sample_submission = pd.read_csv("../input/tabular-playground-series-nov-2021/sample_submission.csv")
sample_submission.target = test_preds
sample_submission.to_csv("submission.csv", index=False)

In [ ]:
sample_submission